<a href="https://colab.research.google.com/github/KateT840/M-computacionales-1/blob/main/Parcial_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install emcee
!pip install corner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 2.1 MB/s eta 0:00:00


In [17]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import emcee
import corner
import os
from google.colab import drive
import scipy.optimize as spo
from scipy.stats import poisson, norm
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [18]:
os.chdir("/content/drive/MyDrive/ColabNotebooks1")
!ls

 DataExponencial1.csv	   DatosRegresion.csv   MinimosCuadratico.txt
 DataExponencial.csv	   Gaussiano.csv        MinimosLineal.txt
'DatosRegresion (1).csv'   MediaMovilData.csv


In [44]:
x1 = np.array([15, 15, 13, 17, 15])
x2 = np.array([18, 20, 16, 21, 15])
_x = (x1, x2)
p0 = [1.5, 0.7, 0.8]

In [45]:
def Likelihood(p,x):
  λ, ε1, ε2 = p
  x1, x2 = x
  t = 10
  L1 = poisson.pmf(x1, mu=λ * t * ε1)
  L2 = poisson.pmf(x2, mu=λ * t * ε2)
  prior_ε1 = norm.pdf(ε1, loc=0.7, scale=0.1)
  prior_ε2 = norm.pdf(ε2, loc=0.8, scale=0.3)
  return np.prod(L1) * np.prod(L2) * prior_ε1 * prior_ε2

In [46]:
Likelihood(p0, _x)

np.float64(3.5424425745972884e-15)

In [47]:
def JointLikelihood(p,x):
  return np.sum(np.log(Likelihood(p,x)))

In [48]:
def LogPrior(p):
    λ, ε1, ε2 = p
    if 0 < λ < 100 and 0 < ε1 < 1 and 0 < ε2 < 1:
        log_prior_ε1 = norm.logpdf(ε1, loc=0.7, scale=0.1)
        log_prior_ε2 = norm.logpdf(ε2, loc=0.8, scale=0.3)
        return log_prior_ε1 + log_prior_ε2
    else:
        return -np.inf

In [49]:
def LogPosterior(p,x):
  LogP = LogPrior(p)
  if not np.isfinite(LogP):
    return -np.inf
  else:
    return LogP + JointLikelihood(p,x)

In [50]:
LogPosterior(p0,_x)

np.float64(-31.60527908196588)